# Xarray-simlab: inspect models

We'll see here how to inspect xarray-simlab models to get information about:

- the model components (processes) and how they depend on each other
- the model state variables and their properties (type, dimensions, description, etc.)
- the model inputs

Let's import xarray-simlab first

In [ ]:
import xsimlab as xs

xs.__version__

Let's import a xarray-simlab model from `fastscape` 

In [ ]:
from fastscape.models import basic_model

## Interactive introspection using the objects `repr`'s

By just typing an object in a code cell, we can display it's `repr` (representation).

### Inspecting models

For a xarray-simlab `Model` object, this gives a list of (ordered) processes and model input variables (i.e., `[in]`) defined in each of those processes. For each input variable it shows a one-line summary with the dimension labels + a (maybe truncated description).

Note: a variable may accepts different numbers of dimensions, e.g., 0D, 1D, 2D, etc. each with their own defined labels. 

In [ ]:
basic_model

### Inspecting model processes

The processes of a model can be accessed either via dict-like access or via Python class attribute-like access. Their `repr` show:

- the name of the corresponding Python class (decorated with `@xs.process`)
- the name of the process in the model
- all variables defined in the process (each with a one line summary: dimension label(s), description, reference variable in case of `xs.foreign` variables)
- the simulation stages that are implemented in the process

Example with dict-like access:

In [ ]:
basic_model['spl']

Example with attribute-like access:

In [ ]:
basic_model.spl

### Inspecting processes and variables not attached to any model

It may be useful to inspect a process and its variables before including it in a xarray-simlab model. Xarray-simlab provides convenient functions for that.

Let's import a process from fastscape

In [ ]:
from fastscape.processes import Escarpment

The function `xs.process_info()` shows similar information than above, without the need here that `Escarpment` be part of a model.

In [ ]:
xs.process_info(Escarpment)

Similarly, `xs.variable_info()` shows information about a variable defined in a process. It shows the complete description as well as all properties.

In [ ]:
xs.variable_info(Escarpment, 'x_left')

## Process and variable docstrings

Another way to inspect processes and variables (whether or not they are part of a model) is by looking at their docstrings. Docstrings are the documentation for Python functions or classes that is directly written in the code (just below the declaration of the class/function). Xarray-simlab also automatically fills the docstrings of process classes, with detailled information extracted from the variables metadata.

This allows to easily generate documentation online, e.g., https://fastscape.readthedocs.io/en/latest/_api_generated/fastscape.processes.FlatSurface.html#fastscape.processes.FlatSurface

In Jupyter notebooks, you can press `shift-tab` to show in a pop-up the docstrings the object where your cursor is on. You can also use `?` or `??`:

In [ ]:
basic_model.init_topography?

In [ ]:
Escarpment?

Variables have also their docstrings (only when the process is in a model):

In [ ]:
basic_model.init_topography.seed?

## Visualize models as graphs

Xarray-simlab models consist of graphs (Directed Acyclic Graph, DAG) of processes.

One process may depend on one or many other processes. Likewise, one process may have one or many dependent processes.

The only constraint is that there cannot by any cycle in the graph. This would mean that a process both computes a value that is required in other processes *and* requires a value computed in those other processes (or processes further in the dependency chain). In this case, it is impossible to determine in which order the processes should be executed so that when a value is needed, it has been already computed (or given as model input).

When a model object is created, Xarray-simlab automatically determines an order in which the processes can be executed (DAG topological sorting), by looking at the intent ('in', 'out', 'inout') of each variable.

Xarray-simlab provides a way to visualize the DAG (each blue node is a process):

In [ ]:
basic_model.visualize()

It is also possible to show the model inputs (in yellow):

In [ ]:
basic_model.visualize(show_inputs=True)

Or all variables (the graph may be not very easy to visualize). In the example below:

- input variables are on the left of a process node and output variables are on the right.
- dashed nodes are foreign variables (reference to a variable declared in another process)

In [ ]:
basic_model.visualize(show_variables=True)

Or only one variable and all its references in all processes:

In [ ]:
basic_model.visualize(show_only_variable=('topography', 'elevation'))

## Other useful ways to extract information from models

It may sometimes be useful to extract information in a format that is more convenient to handle for doing things programmatically. Xarray-simlab provides some convenient functions for that.

For example, to get all model inputs has a list of ``(process_name, var_name)`` tuples:

In [ ]:
basic_model.input_vars

Or as a dictionary

In [ ]:
basic_model.input_vars_dict

Same for all model variables:

In [ ]:
basic_model.all_vars

It is also possible to extract some variable metadata via a model `cache` property

In [ ]:
basic_model.cache[('spl', 'k_coef')]['metadata']